In [1]:
import os 
import sys
from tqdm import tqdm
import cv2
import numpy as np
import json
import skimage.draw
import matplotlib
import matplotlib.pyplot as plt
import random

# Root directory of the project
ROOT_DIR = os.path.abspath('./')
print (ROOT_DIR)
# Import Mask RCNN
sys.path.append(ROOT_DIR) 
from mrcnn.config import Config
from mrcnn import utils
from mrcnn.model import log
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, 'samples/coco/'))
import coco

plt.rcParams['figure.facecolor'] = 'white'

C:\Users\software\Desktop\maskrcnn\Mask_RCNN


C:\Users\software\anaconda3\envs\maskrcnn\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\software\anaconda3\envs\maskrcnn\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\software\anaconda3\envs\maskrcnn\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\software\anaconda3\envs\maskrcnn\lib\site-pack

In [2]:
# ANNOTATIONS_DIR = 'brain-tumor/data/new/annotations/' # directory with annotations for train/val sets
DATASET_DIR = os.path.join(ROOT_DIR, 'data') # directory with image data
MODEL_DIR = 'model_default_logs' 

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

In [3]:
class TextDetectionConfig(Config):
    """Configuration for training on the text detection dataset.
    """
    # Give the configuration a recognizable name
    NAME = 'text_detector'
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + 1  # background + text
    DETECTION_MIN_CONFIDENCE = 0.85    
    STEPS_PER_EPOCH = 100
    LEARNING_RATE = 0.001
    
config = TextDetectionConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.85
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE        

In [4]:
class TextScanDataset(utils.Dataset):
    def load_text_scan(self, dataset_dir, subset):
        """Load a subset of the FarmCow dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add classes. We have only one class to add.
        self.add_class("text", 1, "text")

        # Train or validation dataset?
        assert subset in ["val", "test", "train"]
        
        annotations = []
        path_to_json_dir = os.path.join(DATASET_DIR, subset + '/ocr_results/')
        for file_name in [file for file in os.listdir(path_to_json_dir) if file.endswith('.json')]:
          with open(path_to_json_dir + file_name) as json_file:
            annotation = json.load(json_file)
            interested_area = list(annotation.values())[1][0]
            interested_area["image_path"] = os.path.join(DATASET_DIR, subset + '/documents/', file_name.replace("json", "png"))
            interested_area["image_file_name"] = file_name
            annotations.append(interested_area)      

        # The VIA tool saves images in the JSON even if they don't have any
        # annotations. Skip unannotated images.
        annotations = [a for a in annotations if a['lines']]
        
        for a in annotations:
            width = a["width"]
            height = a["height"]
            image_path= a["image_path"]
            image_file_name = a["image_file_name"]
            polygons = []


            for line in a["lines"]:
                polygons.append(line['boundingBox'])
                for word in line["words"]:
                     polygons.append(word['boundingBox'])


            self.add_image(
                    "text",
                    image_id=image_file_name,  # use file name as a unique image id
                    path=image_path,
                    width=width, 
                    height=height,
                    polygons=polygons
                )
        

    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a farm_cow dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "text":
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
            rr, cc = skimage.draw.polygon( p[1::2],  p[0::2])
            mask[rr, cc, i] = 1

#         # Return mask, and array of class IDs of each instance. Since we have
#         # one class ID only, we return an array of 1s
        return mask.astype(np.bool), np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "text":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)


In [5]:
model = modellib.MaskRCNN(
    mode='training', 
    config=config, 
    model_dir=MODEL_DIR
)

model.load_weights(
    COCO_MODEL_PATH, 
    by_name=True, 
    exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"]
)






Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
box_ind is deprecated, use box_indices instead


In [ ]:
dataset_train = TextScanDataset()
dataset_train.load_text_scan(DATASET_DIR, 'train')
dataset_train.prepare()

dataset_val = TextScanDataset()
dataset_val.load_text_scan(DATASET_DIR, 'val')
dataset_val.prepare()

dataset_test = TextScanDataset()
dataset_test.load_text_scan(DATASET_DIR, 'test')
dataset_test.prepare()

print("Training network heads")
model.train(
    dataset_train, dataset_val,
    learning_rate=config.LEARNING_RATE,
    epochs=10,
    layers='heads',
)

Training network heads

Starting at epoch 0. LR=0.001

Checkpoint Path: model_default_logs\text_detector20220511T1341\mask_rcnn_text_detector_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4      

C:\Users\software\anaconda3\envs\maskrcnn\lib\site-packages\tensorflow\python\ops\gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
C:\Users\software\anaconda3\envs\maskrcnn\lib\site-packages\tensorflow\python\ops\gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
C:\Users\software\anaconda3\envs\maskrcnn\lib\site-packages\tensorflow\python\ops\gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Epoch 1/10
100/100 [==============================] - 1411s - loss: 3.7165 - rpn_class_loss: 0.5415 - rpn_bbox_loss: 1.6858 - mrcnn_class_loss: 0.4329 - mrcnn_bbox_loss: 0.6414 - mrcnn_mask_loss: 0.4148 - val_loss: 2.9419 - val_rpn_class_loss: 0.2138 - val_rpn_bbox_loss: 1.4331 - val_mrcnn_class_loss: 0.4330 - val_mrcnn_bbox_loss: 0.4904 - val_mrcnn_mask_loss: 0.3716
Epoch 2/10
100/100 [==============================] - 857s - loss: 2.8566 - rpn_class_loss: 0.1934 - rpn_bbox_loss: 1.3800 - mrcnn_class_loss: 0.4723 - mrcnn_bbox_loss: 0.4459 - mrcnn_mask_loss: 0.3651 - val_loss: 2.9295 - val_rpn_class_loss: 0.2045 - val_rpn_bbox_loss: 1.4360 - val_mrcnn_class_loss: 0.4479 - val_mrcnn_bbox_loss: 0.4478 - val_mrcnn_mask_loss: 0.3933
Epoch 3/10
 99/100 [============================>.] - ETA: 3s - loss: 2.7050 - rpn_class_loss: 0.1635 - rpn_bbox_loss: 1.3155 - mrcnn_class_l

In [ ]:
# Recreate the model in inference mode
model = modellib.MaskRCNN(
    mode="inference", 
    config=config,
    model_dir=MODEL_DIR
)

# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
model_path = model.find_last()

# Load trained weights
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

In [ ]:
def get_ax(rows=1, cols=1, size=7):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [ ]:
def predict_and_plot_differences(dataset, img_id):
    original_image, image_meta, gt_class_id, gt_box, gt_mask =\
        modellib.load_image_gt(dataset, config, 
                               img_id, use_mini_mask=False)

    results = model.detect([original_image], verbose=1)
    r = results[0]

    visualize.display_differences(
        original_image,
        gt_box, gt_class_id, gt_mask,
        r['rois'], r['class_ids'], r['scores'], r['masks'],
        class_names = ['text'], title="", ax=get_ax(),
        show_mask=True, show_box=True)

In [ ]:
def display_image(dataset, ind):
    plt.figure(figsize=(5,5))
    plt.imshow(dataset.load_image[ind])
    plt.xticks([])
    plt.yticks([])
    plt.title('Original Image')
    plt.show()

In [ ]:
%pylab inline
ind = 50
#display_image(dataset_val, ind)
predict_and_plot_differences(dataset_val, ind)